In [1]:
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
import time
import tkinter
import tkinter as tk
import math
from random import shuffle

In [2]:
class Spieler:
    def __init__(self, n, open_cards, hand):
        self.known = open_cards + hand 
        self.opponent_cards = [[None, None, None] for _ in range(n-1)]
        self.klopfen = False

In [3]:
# Kartenfarbe: 0 = Karo; 1 = Herz; 2 = Pik; 3 = Kreuz
# Kartenbezeichnung: 0 = 7; 1 = 8; 2 = 9; 3 = 10; 4 = Bube, 5 = Dame; 6 = König, 7 = Ass
card_values = [7, 8, 9, 10, 10, 10, 10, 11]
CorX=[290, 420, 550, 290, 420, 550, 30, 160, 290, 550, 680, 810]
CorY=[420, 420, 420, 220, 220, 220, 20, 20, 20, 20, 20, 20]


In [4]:
deck= []
for i in range(4):
    for j in range(8):
        deck.append((i,j))
shuffle(deck)

In [5]:
hand = deck[:3]
oponent_Hand_1 = deck[3:6]
oponent_hand_2 = deck[6:9]
open_cards = deck[9:12]
remainder = deck[12:]

In [6]:
spieler = [Spieler(3, hand, open_cards), Spieler(3, oponent_Hand_1, open_cards), Spieler(3, oponent_hand_2, open_cards)]

In [7]:
def hand_sum(hand):
    points = [0,0,0,0]
    for elem in hand:
        if elem is not None:
            points[elem[0]] += card_values[elem[1]]
    return (np.max(points), points.index(np.max(points)))

In [8]:
def check_for_best_card(hand, open_cards):
    best_card = [hand_sum(hand)[0], None, None]
    for card in open_cards:
        for position in range(3):
            test_hand = hand.copy()
            test_hand[position] = card
            handsum = hand_sum(test_hand)[0]
            if handsum > best_card[0]:
                best_card[0], best_card[1], best_card[2] = handsum, open_cards.index(card), position
    return best_card

In [9]:
def turn(hand, open_cards):
    best_card = check_for_best_card(hand, open_cards)
    if best_card[1] != None:
        hand[best_card[2]], open_cards[best_card[1]] = open_cards[best_card[1]], hand[best_card[2]]
    if hand_sum(hand)[0] >= 31:
        return True
    return 

In [10]:
def update():
    global label
    Hand = hand + open_cards
    cards = [] 
    for card in Hand:
        cards.append(f'PNG-cards-1.3\\{card[0]}{card[1]}.png')
    for j in range(6):
        img = tkinter.PhotoImage(file = cards[j])
        label[j].configure(image=img)
        label[j].image = img


In [11]:
def wahrscheinlichkeit(n, k, n2):
    return math.comb(n, k-n2)/math.comb(n, k)

def possibilities(hand, known, sum):
    n = 0
    deck = [(i, j) for i in range(4) for j in range(8)]
    unknown = [elem for elem in deck if elem not in known]
    for elem in unknown:
        hand[2] = elem
        if hand_sum(hand)[0] >= sum:
            n += 1
    return n

def two_unknown(hand):
    return card_values(hand[0]) + 20 if hand[0] else 30

def check_for_klopfen(hand, open_cards, spieler):
    sum = hand_sum(hand)[0]
    print(sum)
    opponent_pot_sum = [check_for_best_card(elem, open_cards)[0] if elem.count(None) < 2 else two_unknown(elem) for elem in spieler.opponent_cards ]
    print(opponent_pot_sum)
    if sum < min(opponent_pot_sum):
        return False
    print(32-len(spieler.known), spieler.known)
    wsk = [wahrscheinlichkeit(32-len(spieler.known), 1, possibilities(spieler.opponent_cards[i], spieler.known, sum)) for i in range(len(spieler.opponent_cards)) if opponent_pot_sum[i] < sum]
    print(wsk)
    if min(wsk) < 0.5:
        return True
    return False

In [12]:
def who_loses(hand1, hand2, hand3):
    hand_sums = [hand_sum(hand1)[0], hand_sum(hand2)[0], hand_sum(hand3)[0]]
    return hand_sums.index(np.min(hand_sums))


In [13]:

no_action = 0
players = [hand, oponent_Hand_1, oponent_hand_2]

def finish():
    global finish_run
    finish_run = True
    await_next_step.set(1)

window = tkinter.Tk()
window.title("Schwimmen")
window.configure(background="green")
window.attributes("-topmost", True)
window.state("zoomed")
button1 = tkinter.Button(window, text="Nächster\n Zug",command= lambda: await_next_step.set(1))
button1.place(x=800, y=550, width=70, height=40)
button2 = tkinter.Button(window, text="Fertig\n Spielen",command=finish)
button2.place(x=800, y=450, width=70, height=40)
player2background = tk.Button(window, bg="green")
player2background.place(x = 25, y = 15, width = 400, height = 197)
player3background = tk.Button(window, bg="green")
player3background.place(x = 545, y = 15, width = 400, height = 197)
player1background = tk.Button(window, bg = "green")
player1background.place(x = 285, y = 415, width = 400, height = 197)
open_cards_background = tk.Button(window, bg = "green")
open_cards_background.place(x = 285, y = 215, width = 400, height = 195)
playerbackgrounds = [player1background, player2background, player3background]
await_next_step = tk.IntVar()
finish_run = False


def schiebung():
    Schiebung.place_forget()

def schieben():
    open_cards_background.config(bg = "green")

label = []
for i in range(12):
    img = tkinter.PhotoImage(file= "PNG-cards-1.3\\48.png")
    label.append(tkinter.Label(image=img))
    label[i].place(x = CorX[i], y = CorY[i])
    label[i].image = img

Schiebung = tkinter.Button(text="Drei Mal geschoben,\n Karten in der Mittte werden getauscht") 
Schieben = tk.Button(text="Schieben")
update()
for i in range(1000):
    if spieler[i%3].klopfen:
        loser = who_loses(hand, oponent_Hand_1, oponent_hand_2)
        playerbackgrounds[loser].config(bg = "yellow")
        break
    if not check_for_klopfen(players[i%3], open_cards, spieler[i%3]):
        if not finish_run:
            button1.wait_variable(await_next_step)
        open_cards_background.config(bg = "green")
        playerbackgrounds[i%3].config(bg = "red")
        window.update()
        if no_action == 3:
            Schiebung.place(x=325, y=300, width=300, height=50)
            window.after(2500, schiebung)
            print("Schiebung") 
            open_cards, remainder[:3] = remainder[:3], open_cards
            shuffle(remainder)
            no_action = 0
        open_cards_before = open_cards.copy()
        print(players[i%3], i%3 + 1,  hand_sum(players[i%3])[0])
        print(open_cards, "Offenekarten")
        outcome = turn(players[i%3], open_cards)
        if outcome == True:
            loser = who_loses(hand, oponent_Hand_1, oponent_hand_2)
            playerbackgrounds[loser].config(bg = "yellow")
            Hand = hand + open_cards + oponent_Hand_1 + oponent_hand_2
            cards = [] 
            for card in Hand:
                cards.append(f'PNG-cards-1.3\\{card[0]}{card[1]}.png')
            for k in range(12):
                img = tkinter.PhotoImage(file = cards[k])
                label[k].config(image=img)
                label[k].image = img
            break
        if open_cards == open_cards_before:
            open_cards_background.config(bg = "blue")
            no_action += 1
    else:
        spieler[i%3].klopfen = True
    window.update()
    playerbackgrounds[i%3].config(bg = "green")
    window.after(1500,update())
    

window.mainloop()


20
[30, 30]
[(1, 3), (2, 2), (1, 5)] 1 20
[(3, 5), (2, 6), (2, 1)] Offenekarten
10
[30, 30]
[(0, 0), (1, 6), (3, 0)] 2 10
[(3, 5), (2, 6), (2, 1)] Offenekarten
10
[30, 30]
[(3, 3), (0, 3), (1, 0)] 3 10
[(0, 0), (2, 6), (2, 1)] Offenekarten
20
[30, 30]
[(1, 3), (2, 2), (1, 5)] 1 20
[(3, 3), (2, 6), (2, 1)] Offenekarten
17
[30, 30]
[(3, 5), (1, 6), (3, 0)] 2 17
[(3, 3), (2, 6), (2, 1)] Offenekarten
17
[30, 30]
[(0, 0), (0, 3), (1, 0)] 3 17
[(1, 6), (2, 6), (2, 1)] Offenekarten
20
[30, 30]
Schiebung
[(1, 3), (2, 2), (1, 5)] 1 20
[(0, 5), (3, 6), (0, 7)] Offenekarten
27
[30, 30]
[(3, 5), (3, 3), (3, 0)] 2 27
[(0, 5), (3, 6), (0, 7)] Offenekarten
17
[30, 30]
[(0, 0), (0, 3), (1, 0)] 3 17
[(0, 5), (3, 0), (0, 7)] Offenekarten
20
[30, 30]
[(1, 3), (2, 2), (1, 5)] 1 20
[(0, 5), (3, 0), (1, 0)] Offenekarten
30
[30, 30]
26 [(0, 0), (1, 6), (3, 0), (3, 5), (2, 6), (2, 1)]
[]


ValueError: min() arg is an empty sequence

: 